# study2 모델. 차량 파손 여부 탐지

 ## 1. 데이터셋 구축 워크플로우 (Workflow)
 #### 1) 준비 (Preparation)
  - 입력 데이터:
    - Damaged 폴더 (파손 차량 이미지 + 파손 위치 라벨 JSON)
    - Normal 폴더 (정상 차량 이미지)
 - 도구 (Tool):
    - Study 1 모델: 이미 학습된 차량 탐지 모델 (yolov8_fine_tuning.pt)
 #### 2) 데이터 분할 (Splitting)
  - 비율: 전체 데이터를 Train (70%) : Val (20%) : Test (10%) 비율로 무작위 분할
  - 목적: 학습용, 검증용, 평가용 데이터를 미리 나누어 데이터 유출(Data Leakage)을 방지
 #### 3) 핵심 처리 과정 (Core Process) - 반복문(Loop)
  - 차량 탐지 (Detect Vehicle):
    - 1단계 모델을 사용하여 이미지에서 **차량의위치BBox**를 찾음
    - (만약 차량을 못 찾으면? → 원본 이미지 전체를 사용하도록 Fallback)
 - 영역 자르기 (Crop with Margin):
    - 차량 BBox에 **여유공간(Margin15%)** 을 더해서 이미지를 잘라냄.
    - 이때, 이미지 밖으로 나가는 좌표는 **Clamp(울타리)** 처리하여 이미지 안쪽으로 맞춤
 - 크기 조정 (Resize):
    - 잘라낸 이미지를 모델 입력 크기(예: 640x640 또는 1024x1024)로 변경
 - 라벨 변환 (Label Remapping):
    - Damaged: 원본 이미지 기준의 파손 좌표 (x, y)를 Crop된 이미지 기준의 상대 좌표로 변환.(이때도 좌표가 튀지 않게 Clamp 적용 필수)
    - Normal: 파손이 없으므로 **빈 텍스트 파일.txt**을 생성 (Negative Sample 학습용)
 #### 4) 저장 (Saving)
 - 변환된 이미지와 라벨(.txt)을 YOLO 학습 포맷에 맞춰 저장
    - images/train,
    - labels/trainimages/val,
    - labels/valimages/test, labels/test
 - **클래스 통합**: 데이터 복잡도를 줄이기 위해 모든 파손 종류(Scratch, Dent 등)를 'Damage' (ID: 0) 하나로 통합
 #### 5) 검증 (Verification)
 - 시각화: 변환된 데이터(Test Set) 중 3장을 무작위로 뽑아, 파손 박스가 차량 위에 정확히 그려졌는지 눈으로 확인

# 데이터셋 구축 및 분할

In [ ]:
!pip install ultralytics tqdm

# [Cell 1] 라이브러리 임포트 및 설정
import os
import glob
import shutil
import json
import random
import cv2
import yaml
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from google.colab import drive

# 구글 드라이브 마운트
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

print("✅ 환경 설정 완료")

# =========================================================
# [설정] 경로 및 파라미터 (환경에 맞게 수정 필요)
# =========================================================
# 1. 원본 데이터 경로
SOURCE_ROOT = "/content/drive/MyDrive/03. HDMF/(share)HDMF_AUTO_SPOKE/DATA/(share)2026_ImageDetectionStudy_No_resizing"

# 2. 저장할 데이터셋 경로
DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DAMAGE_DETECTION_CLASS_ALL"

# 3. 1단계 차량 탐지 모델 경로 (★ 중요: 실제 pt 파일 경로로 수정해주세요)
# 예: .../runs/detect/train/weights/best.pt
MODEL_PATH = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION/FINE_TUNING_MODEL/yolov8m_fine_tuning_3rd/weights/best.pt"

# 4. 파라미터
IMG_SIZE = 640       # 2단계 모델 입력 사이즈 (리사이징)
MARGIN_RATIO = 0.15  # 차량 Crop 시 여유 공간 (15%)
SEED = 42

random.seed(SEED)
np.random.seed(SEED)

print("✅ 설정 완료")

In [ ]:
# [Cell 2] 유틸리티 함수 정의 (Clamp, 좌표 변환)

def clamp(value, min_val, max_val):
    """값을 [min_val, max_val] 범위 내로 고정 (Clamp 로직)"""
    return max(min_val, min(value, max_val))

def get_vehicle_crop_box(image, model, margin=0.15):
    """
    이미지에서 차량을 탐지하고, Margin을 포함한 Crop 좌표(x1, y1, x2, y2)를 반환.
    탐지 실패 시 전체 이미지 좌표 반환 (Fallback).
    """
    h_img, w_img = image.shape[:2]

    # 1단계 모델 추론
    results = model(image, verbose=False, conf=0.25)

    best_box = None
    max_area = 0

    # 가장 큰 차량 박스 선택 (여러 대일 경우 메인 차량 타겟팅)
    for r in results:
        for box in r.boxes:
            # box.xyxy: [x1, y1, x2, y2]
            coords = box.xyxy[0].cpu().numpy()
            area = (coords[2] - coords[0]) * (coords[3] - coords[1])
            if area > max_area:
                max_area = area
                best_box = coords

    if best_box is None:
        # Fallback: 차량 미탐지 시 전체 이미지 사용 (중앙 Crop 효과 등을 위해)
        return 0, 0, w_img, h_img, False # False indicates detection failed

    # Margin 적용
    x1, y1, x2, y2 = best_box
    w_box = x2 - x1
    h_box = y2 - y1

    m_w = w_box * margin
    m_h = h_box * margin

    # 이미지 경계를 벗어나지 않도록 Clamp 적용
    nx1 = int(clamp(x1 - m_w, 0, w_img))
    ny1 = int(clamp(y1 - m_h, 0, h_img))
    nx2 = int(clamp(x2 + m_w, 0, w_img))
    ny2 = int(clamp(y2 + m_h, 0, h_img))

    return nx1, ny1, nx2, ny2, True

def process_and_save(src_img_path, src_json_path, dst_img_path, dst_txt_path, model):
    """단일 이미지 처리 및 저장"""
    img = cv2.imread(src_img_path)
    if img is None: return False

    h_org, w_org = img.shape[:2]

    # 1. 차량 Crop 좌표 계산
    cx1, cy1, cx2, cy2, detected = get_vehicle_crop_box(img, model, MARGIN_RATIO)
    crop_w = cx2 - cx1
    crop_h = cy2 - cy1

    if crop_w <= 0 or crop_h <= 0: return False # 예외 처리

    # 2. 이미지 Crop & Resize
    crop_img = img[cy1:cy2, cx1:cx2]
    resized_img = cv2.resize(crop_img, (IMG_SIZE, IMG_SIZE))

    # 3. 라벨 처리 (Normal은 빈 리스트, Damaged는 JSON 파싱)
    yolo_labels = []

    if src_json_path and os.path.exists(src_json_path):
        with open(src_json_path, 'r') as f:
            data = json.load(f)

        for ann in data.get('annotations', []):
            # JSON format: bbox [x_min, y_min, width, height] (Global)
            gx, gy, gw, gh = ann['bbox']

            # Global xyxy
            gx1, gy1 = gx, gy
            gx2, gy2 = gx + gw, gy + gh

            # Remapping: Global -> Crop Local
            lx1 = gx1 - cx1
            ly1 = gy1 - cy1
            lx2 = gx2 - cx1
            ly2 = gy2 - cy1

            # Clamp Logic: Crop 영역 밖의 파손 부위 잘라내기
            lx1 = clamp(lx1, 0, crop_w)
            ly1 = clamp(ly1, 0, crop_h)
            lx2 = clamp(lx2, 0, crop_w)
            ly2 = clamp(ly2, 0, crop_h)

            # 유효성 검사 (너비나 높이가 너무 작아졌으면 스킵)
            lw = lx2 - lx1
            lh = ly2 - ly1
            if lw < 1 or lh < 1:
                continue

            # YOLO Format Conversion (Normalized 0~1)
            # Resize 했으므로 비율은 Crop 이미지 기준과 동일
            x_center = (lx1 + lw / 2) / crop_w
            y_center = (ly1 + lh / 2) / crop_h
            w_norm = lw / crop_w
            h_norm = lh / crop_h

            # Class ID: 0 (Damage 통합)
            yolo_labels.append(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")

    # 4. 저장
    cv2.imwrite(dst_img_path, resized_img)

    with open(dst_txt_path, 'w') as f:
        f.write("\n".join(yolo_labels))

    return True

print("✅ 함수 정의 완료")

## 데이터셋 구축 메인 프로세스 (반복 x)

In [ ]:
# [Cell 3] 데이터셋 구축 메인 프로세스
def build_dataset():
    # 0. 디렉토리 초기화 및 생성
    if os.path.exists(DATASET_ROOT):
        shutil.rmtree(DATASET_ROOT)

    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(DATASET_ROOT, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(DATASET_ROOT, 'labels', split), exist_ok=True)

    # 1. 모델 로드
    if not os.path.exists(MODEL_PATH):
        print(f"❌ 오류: 모델 파일이 없습니다. 경로를 확인하세요: {MODEL_PATH}")
        return
    print(f"🚀 모델 로드 중: {os.path.basename(MODEL_PATH)}")
    model = YOLO(MODEL_PATH)

    # 2. 파일 리스트 수집
    print("📂 파일 리스트 수집 중...")

    # Damaged Data (Positive)
    damaged_img_dir = os.path.join(SOURCE_ROOT, "damaged", "images")
    damaged_lbl_dir = os.path.join(SOURCE_ROOT, "damaged", "labels")
    damaged_files = glob.glob(os.path.join(damaged_img_dir, "*.jpg")) + \
                    glob.glob(os.path.join(damaged_img_dir, "*.png"))

    # Normal Data (Negative)
    normal_img_dir = os.path.join(SOURCE_ROOT, "normal", "images")
    normal_files = glob.glob(os.path.join(normal_img_dir, "*.jpg")) + \
                   glob.glob(os.path.join(normal_img_dir, "*.png"))

    # 3. 데이터 분할 (7:2:1)
    def split_data(file_list):
        train, test = train_test_split(file_list, test_size=0.1, random_state=SEED)
        train, val = train_test_split(train, test_size=0.2222, random_state=SEED) # 0.9 * 0.2222 ≈ 0.2
        return train, val, test

    d_train, d_val, d_test = split_data(damaged_files)
    n_train, n_val, n_test = split_data(normal_files)

    # 4. 처리 루프
    tasks = [
        ('train', d_train, 'damaged'), ('val', d_val, 'damaged'), ('test', d_test, 'damaged'),
        ('train', n_train, 'normal'),  ('val', n_val, 'normal'),  ('test', n_test, 'normal')
    ]

    total_stats = {'train': 0, 'val': 0, 'test': 0}

    print("⚡ 데이터 전처리 및 Crop 시작 (시간이 다소 소요됩니다)...")

    for split, files, kind in tasks:
        for file_path in tqdm(files, desc=f"[{split}] {kind}"):
            filename = os.path.basename(file_path)
            basename = os.path.splitext(filename)[0]

            dst_img = os.path.join(DATASET_ROOT, 'images', split, filename)
            dst_txt = os.path.join(DATASET_ROOT, 'labels', split, basename + ".txt")

            json_path = None
            if kind == 'damaged':
                json_path = os.path.join(damaged_lbl_dir, basename + ".json")

            success = process_and_save(file_path, json_path, dst_img, dst_txt, model)
            if success:
                total_stats[split] += 1

    # 5. YAML 파일 생성
    yaml_content = f"""
path: {DATASET_ROOT}
train: images/train
val: images/val
test: images/test

nc: 1
names: ['Damage']
"""
    with open(os.path.join(DATASET_ROOT, "data_damage.yaml"), "w") as f:
        f.write(yaml_content)

    print("\n" + "="*50)
    print("📊 최종 데이터셋 생성 현황")
    print(f"   Train : {total_stats['train']} 장")
    print(f"   Val   : {total_stats['val']} 장")
    print(f"   Test  : {total_stats['test']} 장")
    print(f"   Total : {sum(total_stats.values())} 장")
    print(f"   YAML  : {os.path.join(DATASET_ROOT, 'data_damage.yaml')}")
    print("="*50)

# 실행
build_dataset()

## 시각화 (검증)

In [ ]:
# [Cell 4] 시각화 (검증)
def visualize_random_samples(num_samples=3):
    """
    생성된 데이터셋(Test Set) 중 Damaged 라벨이 있는 이미지를 랜덤으로 골라 시각화
    """
    test_img_dir = os.path.join(DATASET_ROOT, 'images', 'test')
    test_lbl_dir = os.path.join(DATASET_ROOT, 'labels', 'test')

    # 1. 라벨 파일이 있는(내용이 있는) 파일 찾기
    candidates = []
    for txt_file in os.listdir(test_lbl_dir):
        txt_path = os.path.join(test_lbl_dir, txt_file)
        if os.path.getsize(txt_path) > 0: # 내용이 있으면 Damage가 있는 것
            candidates.append(txt_file)

    if not candidates:
        print("시각화할 파손 데이터가 없습니다.")
        return

    # 2. 랜덤 샘플링
    samples = random.sample(candidates, min(len(candidates), num_samples))

    plt.figure(figsize=(15, 5))

    for i, txt_name in enumerate(samples):
        basename = os.path.splitext(txt_name)[0]
        img_name = basename + ".jpg" # 확장자는 jpg라고 가정 (또는 확인 필요)
        if not os.path.exists(os.path.join(test_img_dir, img_name)):
            img_name = basename + ".png"

        img_path = os.path.join(test_img_dir, img_name)
        txt_path = os.path.join(test_lbl_dir, txt_name)

        # 이미지 로드
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        # 박스 그리기
        with open(txt_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                cls, cx, cy, bw, bh = map(float, line.strip().split())

                # YOLO -> Pixel
                x1 = int((cx - bw/2) * w)
                y1 = int((cy - bh/2) * h)
                x2 = int((cx + bw/2) * w)
                y2 = int((cy + bh/2) * h)

                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(img, "Damage", (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        plt.subplot(1, num_samples, i+1)
        plt.imshow(img)
        plt.title(f"Sample {i+1}\n{basename}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# 실행
print("🔍 생성된 데이터셋 검증 (Random 3 Samples)")

In [ ]:
visualize_random_samples()

In [ ]:
### train시 1시간 정도 소모 됩니다. colab환경에서 아래를 browser의 console 에서 붙여 넣기가 필요할 수 있습니다.
### shift+cntr+i 로 browser console 열기
# https://github.com/chulminkw/DLCV/blob/master/data/util/colab_autoclick.js
'''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''

# 모델 학습 1차

In [ ]:
# [Cell 1] 라이브러리 임포트 및 학습 설정
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from ultralytics import YOLO
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tqdm import tqdm

# =========================================================
# [설정] 경로 (Step 2에서 생성한 경로와 일치해야 함)
# =========================================================
DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DAMAGE_DETECTION_CLASS_ALL"
PROJECT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION"

# YAML 파일 경로
YAML_PATH = os.path.join(DATASET_ROOT, 'data_damage.yaml')

# 저장 경로 초기화
SAVE_DIR = os.path.join(PROJECT_ROOT, "TRAIN_RESULT")
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

print(f"📂 데이터셋 경로: {DATASET_ROOT}")
print(f"💾 결과 저장 경로: {SAVE_DIR}")

In [ ]:
# [Cell 2] YOLOv8 모델 학습 (Training)

def train_model():
    # 1. 모델 로드 (파손은 미세한 특징이 중요하므로 'm' 모델 권장)
    # 기존 pt 파일이 있다면 그것을 로드해서 Fine-tuning, 없다면 yolov8m.pt 다운로드
    model = YOLO('yolov8m.pt')

    print("🔥 학습 시작 (Damage Detection)...")

    # 2. 학습 실행
    model.train(
        data=YAML_PATH,
        epochs=100,            # 50 Epoch (데이터 양에 따라 조절)
        imgsz=640,            # Crop된 이미지이므로 640이면 충분
        batch=16,
        patience=10,          # 10번 동안 성능 향상 없으면 조기 종료
        project=SAVE_DIR,     # 저장할 프로젝트 폴더
        name='yolov8m_damage_class_all_1st',# 저장할 하위 폴더 이름
        exist_ok=True,
        optimizer='AdamW',
        lr0=1e-4,             # Fine-tuning이므로 학습률을 조금 낮게 설정
        close_mosaic=10,      # 마지막 10 epoch은 Mosaic Augmentation 끄기 (정밀도 향상)
        verbose=True
    )

    print("🎉 학습 완료!")
    return os.path.join(SAVE_DIR, 'yolov8m_damage', 'weights', 'best.pt')

# 학습 실행
best_model_path = train_model()
print(f"🏆 Best Model Path: {best_model_path}")

## 성능 테스트(test)

In [ ]:
# [Cell 3] 성능 평가 (Detection & Classification)

best_model_path = '/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/TRAIN_RESULT/yolov8m_damage_class_all_1st/weights/best.pt'
RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"
os.makedirs(RESULT_ROOT, exist_ok=True)

def evaluate_performance(model_path):
    if not os.path.exists(model_path):
        print(f"❌ 모델 파일이 없습니다: {model_path}")
        return

    print(f"🚀 평가 시작...")
    print(f"   - 모델: {model_path}")

    model = YOLO(model_path)

    # ---------------------------------------------------------
    # 1. 객체 탐지 성능 평가 (Detection Metrics: mAP)
    # ---------------------------------------------------------
    print("\n📊 [1] BBox 탐지 성능 (mAP) 계산 중...")
    # split='test'로 설정하여 Test Set에 대해 평가
    metrics = model.val(data=YAML_PATH, split='test', verbose=False)

    map50 = metrics.box.map50
    map5095 = metrics.box.map

    # ---------------------------------------------------------
    # 2. 이미지 분류 성능 & 속도 평가 (Classification & Speed)
    # ---------------------------------------------------------
    print("📊 [2] 이미지 분류 성능 및 추론 속도 측정 중...")

    TEST_IMG_DIR = os.path.join(DATASET_ROOT, 'images', 'test')
    TEST_LBL_DIR = os.path.join(DATASET_ROOT, 'labels', 'test')

    image_files = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    results_list = []
    speed_stats = []

    for file in tqdm(image_files, desc="Inference"):
        img_path = os.path.join(TEST_IMG_DIR, file)
        txt_file = os.path.splitext(file)[0] + ".txt"
        lbl_path = os.path.join(TEST_LBL_DIR, txt_file)

        # A. 정답 확인 (라벨 파일 내용 유무)
        true_label = 0
        if os.path.exists(lbl_path) and os.path.getsize(lbl_path) > 0:
            true_label = 1 # Damaged

        # B. 모델 추론
        results = model(img_path, conf=0.25, verbose=False)

        # C. 속도 측정
        speed_info = results[0].speed
        total_time_ms = speed_info['preprocess'] + speed_info['inference'] + speed_info['postprocess']
        speed_stats.append(total_time_ms)

        # D. 예측 라벨 (박스 유무)
        pred_label = 0
        if len(results[0].boxes) > 0:
            pred_label = 1 # Damaged

        results_list.append({
            "filename": file,
            "true_label": true_label,
            "pred_label": pred_label,
            "is_correct": (true_label == pred_label)
        })

    df = pd.DataFrame(results_list)

    # =========================================================
    # [최종 분석 결과 리포트]
    # =========================================================
    print("\n" + "="*50)
    print("📊 [최종 분석 결과 - Fine-tuned Model]")
    print("="*50)

    # 1. mAP 결과 출력
    print(f"🎯 1. 객체 탐지 성능 (Detection Metrics):")
    print(f"   - mAP@50    : {map50:.4f}")
    print(f"   - mAP@50-95 : {map5095:.4f}")

    # 2. 정확도 출력
    acc = accuracy_score(df['true_label'], df['pred_label'])
    print(f"\n✅ 2. 분류 정확도 (Accuracy): {acc:.4f} ({acc*100:.2f}%)")

    # 3. 속도 출력
    if speed_stats:
        avg_time = np.mean(speed_stats)
        min_time = np.min(speed_stats)
        max_time = np.max(speed_stats)
        fps = 1000 / avg_time if avg_time > 0 else 0

        print(f"\n⚡ 3. 추론 속도 (Inference Speed):")
        print(f"   - 평균 소요 시간 : {avg_time:.2f} ms/장")
        print(f"   - 최소 소요 시간 : {min_time:.2f} ms")
        print(f"   - 최대 소요 시간 : {max_time:.2f} ms")
        print(f"   - 처리량 (FPS)   : {fps:.2f} FPS")

    # 4. 상세 리포트
    print(f"\n📝 4. 상세 분류 리포트:")
    print(classification_report(df['true_label'], df['pred_label'], target_names=['Normal', 'Damaged']))

    # 5. 혼동 행렬 시각화
    plt.figure(figsize=(6, 5))
    cm = confusion_matrix(df['true_label'], df['pred_label'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred: Normal', 'Pred: Damaged'],
                yticklabels=['True: Normal', 'True: Damaged'])
    plt.title(f'Confusion Matrix\n(Damage Detection)')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

    # 결과 저장
    save_path = os.path.join(RESULT_ROOT, "inference_damage_test_results_1st_025.csv")
    df.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 전체 실행 결과 저장 완료: {save_path}")

# 평가 실행
evaluate_performance(best_model_path)


## 오탐 대상 시각화

In [ ]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO

# =========================================================
# [설정] 경로 (이전 단계와 동일하게 설정)
# =========================================================
DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DAMAGE_DETECTION_CLASS_ALL"
PROJECT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION"
SAVE_DIR = os.path.join(PROJECT_ROOT, "TRAIN_RESULT")
RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"


# 모델 및 결과 파일 경로
MODEL_PATH = os.path.join(SAVE_DIR, 'yolov8m_damage_class_all_1st', 'weights', 'best.pt')
RESULT_CSV_PATH = os.path.join(RESULT_ROOT, "inference_damage_test_results_1st_025.csv")

# 테스트 이미지 및 라벨 경로
TEST_IMG_DIR = os.path.join(DATASET_ROOT, 'images', 'test')
TEST_LBL_DIR = os.path.join(DATASET_ROOT, 'labels', 'test')

def visualize_failures_batch(start_index=0, batch_size=3):
    """
    오탐 데이터를 start_index부터 batch_size만큼 시각화하는 함수
    """
    # 1. 결과 CSV 로드 및 오탐 필터링
    if not os.path.exists(RESULT_CSV_PATH):
        print(f"❌ 결과 파일이 없습니다: {RESULT_CSV_PATH}")
        return

    df = pd.read_csv(RESULT_CSV_PATH)
    failures = df[df['is_correct'] == False] # 오탐만 추출

    total_failures = len(failures)
    if total_failures == 0:
        print("🎉 오탐 데이터가 없습니다! 완벽합니다.")
        return

    # 인덱스 범위 체크
    if start_index >= total_failures:
        print(f"⚠️ 더 이상 확인할 오탐 데이터가 없습니다. (총 {total_failures}건)")
        return

    end_index = min(start_index + batch_size, total_failures)
    target_samples = failures.iloc[start_index : end_index]

    print(f"🔍 오탐 데이터 시각화: {start_index+1} ~ {end_index} / 총 {total_failures}건")

    # 2. 모델 로드 (박스 시각화용)
    model = YOLO(MODEL_PATH)

    # 3. 시각화
    plt.figure(figsize=(15, 6 * len(target_samples)))

    for i, (_, row) in enumerate(target_samples.iterrows()):
        filename = row['filename']
        true_label = row['true_label'] # 0: Normal, 1: Damaged
        img_path = os.path.join(TEST_IMG_DIR, filename)
        lbl_path = os.path.join(TEST_LBL_DIR, os.path.splitext(filename)[0] + ".txt")

        # 이미지 로드
        img = cv2.imread(img_path)
        if img is None: continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        # A. 정답(Ground Truth) 박스 그리기 (초록색)
        if os.path.exists(lbl_path):
            with open(lbl_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    cls, cx, cy, bw, bh = map(float, line.strip().split())
                    x1 = int((cx - bw/2) * w)
                    y1 = int((cy - bh/2) * h)
                    x2 = int((cx + bw/2) * w)
                    y2 = int((cy + bh/2) * h)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    cv2.putText(img, "GT", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # B. 모델 예측(Prediction) 박스 그리기 (빨간색)
        # 시각화를 위해 다시 추론 수행
        results = model(img_path, conf=0.25, verbose=False)
        for r in results:
            for box in r.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
                cv2.putText(img, f"Pred {conf:.2f}", (x1, y2+25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        # 타이틀 설정
        str_true = "Damaged" if true_label == 1 else "Normal"
        str_pred = "Normal" if true_label == 1 else "Damaged" # 오탐이므로 반대
        fail_type = "False Negative (미탐)" if true_label == 1 else "False Positive (오탐/과탐)"

        plt.subplot(len(target_samples), 1, i+1)
        plt.imshow(img)
        plt.title(f"[{start_index+i+1}] {filename}\nType: {fail_type} | GT: {str_true} (Green) vs Pred: {str_pred} (Red)",
                  fontsize=14, color='red', fontweight='bold')
        plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# 모델 및 결과 파일 경로
MODEL_PATH = os.path.join(SAVE_DIR, 'yolov8m_damage_class_all_1st', 'weights', 'best.pt')
RESULT_CSV_PATH = os.path.join(RESULT_ROOT, "inference_damage_test_results_1st_025.csv")

visualize_failures_batch(start_index=0, batch_size=20)

In [ ]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO

# =========================================================
# [설정] 경로 (이전 단계와 동일)
# =========================================================
DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DAMAGE_DETECTION_CLASS_ALL"
PROJECT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION"
SAVE_DIR = os.path.join(PROJECT_ROOT, "TRAIN_RESULT")

# 모델 및 결과 파일 경로
MODEL_PATH = os.path.join(SAVE_DIR, 'yolov8m_damage_class_all_1st', 'weights', 'best.pt')
RESULT_CSV_PATH = os.path.join(RESULT_ROOT, "inference_damage_test_results_1st.csv")

# 테스트 이미지 및 라벨 경로
TEST_IMG_DIR = os.path.join(DATASET_ROOT, 'images', 'test')
TEST_LBL_DIR = os.path.join(DATASET_ROOT, 'labels', 'test')

def visualize_tp_batch(start_index=0, batch_size=3):
    """
    정탐(True Positive) 데이터를 start_index부터 batch_size만큼 시각화하는 함수
    """
    # 1. 결과 CSV 로드
    if not os.path.exists(RESULT_CSV_PATH):
        print(f"❌ 결과 파일이 없습니다: {RESULT_CSV_PATH}")
        return

    df = pd.read_csv(RESULT_CSV_PATH)

    # 2. TP 필터링 (정답이 Damaged(1)이고, 예측도 Damaged(1)인 경우)
    # is_correct가 True이면서 true_label이 1인 것
    tp_cases = df[(df['is_correct'] == True) & (df['true_label'] == 1)]

    total_tp = len(tp_cases)
    if total_tp == 0:
        print("⚠️ 정탐(True Positive) 데이터가 하나도 없습니다.")
        return

    # 인덱스 범위 체크
    if start_index >= total_tp:
        print(f"⚠️ 더 이상 확인할 TP 데이터가 없습니다. (총 {total_tp}건)")
        return

    end_index = min(start_index + batch_size, total_tp)
    target_samples = tp_cases.iloc[start_index : end_index]

    print(f"✅ 정탐(TP) 데이터 시각화: {start_index+1} ~ {end_index} / 총 {total_tp}건")

    # 3. 모델 로드
    model = YOLO(MODEL_PATH)

    # 4. 시각화
    plt.figure(figsize=(15, 6 * len(target_samples)))

    for i, (_, row) in enumerate(target_samples.iterrows()):
        filename = row['filename']
        img_path = os.path.join(TEST_IMG_DIR, filename)
        lbl_path = os.path.join(TEST_LBL_DIR, os.path.splitext(filename)[0] + ".txt")

        # 이미지 로드
        img = cv2.imread(img_path)
        if img is None: continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        # A. 정답(Ground Truth) 박스 그리기 (초록색)
        if os.path.exists(lbl_path):
            with open(lbl_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    cls, cx, cy, bw, bh = map(float, line.strip().split())
                    x1 = int((cx - bw/2) * w)
                    y1 = int((cy - bh/2) * h)
                    x2 = int((cx + bw/2) * w)
                    y2 = int((cy + bh/2) * h)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    cv2.putText(img, "GT", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # B. 모델 예측(Prediction) 박스 그리기 (빨간색)
        results = model(img_path, conf=0.25, verbose=False)
        for r in results:
            for box in r.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
                cv2.putText(img, f"Pred {conf:.2f}", (x1, y2+25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        # 타이틀 설정
        plt.subplot(len(target_samples), 1, i+1)
        plt.imshow(img)
        plt.title(f"[{start_index+i+1}] {filename}\nType: True Positive (정탐) | GT: Damaged (Green) vs Pred: Damaged (Red)",
                  fontsize=14, color='green', fontweight='bold')
        plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# =========================================================
# 실행: 첫 번째 3장 확인
# =========================================================
visualize_tp_batch(start_index=9, batch_size=3)

## 성능 테스트(conf 0.25->0.1)

In [ ]:
# [Cell 3] 성능 평가 (Detection & Classification)

best_model_path = '/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/TRAIN_RESULT/yolov8m_damage_class_all_1st/weights/best.pt'
RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"
os.makedirs(RESULT_ROOT, exist_ok=True)

def evaluate_performance(model_path):
    if not os.path.exists(model_path):
        print(f"❌ 모델 파일이 없습니다: {model_path}")
        return

    print(f"🚀 평가 시작...")
    print(f"   - 모델: {model_path}")

    model = YOLO(model_path)

    # ---------------------------------------------------------
    # 1. 객체 탐지 성능 평가 (Detection Metrics: mAP)
    # ---------------------------------------------------------
    print("\n📊 [1] BBox 탐지 성능 (mAP) 계산 중...")
    # split='test'로 설정하여 Test Set에 대해 평가
    metrics = model.val(data=YAML_PATH, split='test', verbose=False)

    map50 = metrics.box.map50
    map5095 = metrics.box.map

    # ---------------------------------------------------------
    # 2. 이미지 분류 성능 & 속도 평가 (Classification & Speed)
    # ---------------------------------------------------------
    print("📊 [2] 이미지 분류 성능 및 추론 속도 측정 중...")

    TEST_IMG_DIR = os.path.join(DATASET_ROOT, 'images', 'test')
    TEST_LBL_DIR = os.path.join(DATASET_ROOT, 'labels', 'test')

    image_files = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    results_list = []
    speed_stats = []

    for file in tqdm(image_files, desc="Inference"):
        img_path = os.path.join(TEST_IMG_DIR, file)
        txt_file = os.path.splitext(file)[0] + ".txt"
        lbl_path = os.path.join(TEST_LBL_DIR, txt_file)

        # A. 정답 확인 (라벨 파일 내용 유무)
        true_label = 0
        if os.path.exists(lbl_path) and os.path.getsize(lbl_path) > 0:
            true_label = 1 # Damaged

        # B. 모델 추론
        results = model(img_path, conf=0.1, verbose=False)

        # C. 속도 측정
        speed_info = results[0].speed
        total_time_ms = speed_info['preprocess'] + speed_info['inference'] + speed_info['postprocess']
        speed_stats.append(total_time_ms)

        # D. 예측 라벨 (박스 유무)
        pred_label = 0
        if len(results[0].boxes) > 0:
            pred_label = 1 # Damaged

        results_list.append({
            "filename": file,
            "true_label": true_label,
            "pred_label": pred_label,
            "is_correct": (true_label == pred_label)
        })

    df = pd.DataFrame(results_list)

    # =========================================================
    # [최종 분석 결과 리포트]
    # =========================================================
    print("\n" + "="*50)
    print("📊 [최종 분석 결과 - Fine-tuned Model]")
    print("="*50)

    # 1. mAP 결과 출력
    print(f"🎯 1. 객체 탐지 성능 (Detection Metrics):")
    print(f"   - mAP@50    : {map50:.4f}")
    print(f"   - mAP@50-95 : {map5095:.4f}")

    # 2. 정확도 출력
    acc = accuracy_score(df['true_label'], df['pred_label'])
    print(f"\n✅ 2. 분류 정확도 (Accuracy): {acc:.4f} ({acc*100:.2f}%)")

    # 3. 속도 출력
    if speed_stats:
        avg_time = np.mean(speed_stats)
        min_time = np.min(speed_stats)
        max_time = np.max(speed_stats)
        fps = 1000 / avg_time if avg_time > 0 else 0

        print(f"\n⚡ 3. 추론 속도 (Inference Speed):")
        print(f"   - 평균 소요 시간 : {avg_time:.2f} ms/장")
        print(f"   - 최소 소요 시간 : {min_time:.2f} ms")
        print(f"   - 최대 소요 시간 : {max_time:.2f} ms")
        print(f"   - 처리량 (FPS)   : {fps:.2f} FPS")

    # 4. 상세 리포트
    print(f"\n📝 4. 상세 분류 리포트:")
    print(classification_report(df['true_label'], df['pred_label'], target_names=['Normal', 'Damaged']))

    # 5. 혼동 행렬 시각화
    plt.figure(figsize=(6, 5))
    cm = confusion_matrix(df['true_label'], df['pred_label'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred: Normal', 'Pred: Damaged'],
                yticklabels=['True: Normal', 'True: Damaged'])
    plt.title(f'Confusion Matrix\n(Damage Detection)')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

    # 결과 저장
    save_path = os.path.join(RESULT_ROOT, "inference_damage_test_results_1st_010.csv")
    df.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 전체 실행 결과 저장 완료: {save_path}")

# 평가 실행
evaluate_performance(best_model_path)


In [ ]:
# 모델 및 결과 파일 경로
MODEL_PATH = os.path.join(SAVE_DIR, 'yolov8m_damage_class_all_1st', 'weights', 'best.pt')
RESULT_CSV_PATH = os.path.join(RESULT_ROOT, "inference_damage_test_results_1st_010.csv")

visualize_failures_batch(start_index=0, batch_size=13)

# 모델 학습 2차
 - IMG_SIZE = 1024
 - model = yolov8x
 - close_mosaic=15

In [ ]:
# [Cell 1] 라이브러리 임포트 및 학습 설정
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from ultralytics import YOLO
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tqdm import tqdm

# =========================================================
# [설정] 경로 (Step 2에서 생성한 경로와 일치해야 함)
# =========================================================
DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DAMAGE_DETECTION_CLASS_ALL"
PROJECT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION"

# YAML 파일 경로
YAML_PATH = os.path.join(DATASET_ROOT, 'data_damage.yaml')

# 저장 경로 초기화
SAVE_DIR = os.path.join(PROJECT_ROOT, "TRAIN_RESULT")
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

print(f"📂 데이터셋 경로: {DATASET_ROOT}")
print(f"💾 결과 저장 경로: {SAVE_DIR}")

In [ ]:
# [Cell 2] YOLOv8 모델 학습 (Training)

def train_model():
    # 1. 모델 로드 (파손은 미세한 특징이 중요하므로 'm' 모델 권장)
    # 기존 pt 파일이 있다면 그것을 로드해서 Fine-tuning, 없다면 yolov8m.pt 다운로드
    model = YOLO('yolov8x.pt')

    print("🔥 학습 시작 (Damage Detection)...")

    # 2. 학습 실행
    model.train(
        data=YAML_PATH,
        epochs=100,            # 50 Epoch (데이터 양에 따라 조절)
        imgsz=1024,            # Crop된 이미지이므로 640이면 충분
        batch=16,
        patience=10,          # 10번 동안 성능 향상 없으면 조기 종료
        project=SAVE_DIR,     # 저장할 프로젝트 폴더
        name='yolov8x_damage_class_all_2nd',# 저장할 하위 폴더 이름
        exist_ok=True,
        optimizer='AdamW',
        lr0=1e-4,             # Fine-tuning이므로 학습률을 조금 낮게 설정
        close_mosaic=20,      # 마지막 10 epoch은 Mosaic Augmentation 끄기 (정밀도 향상)
        verbose=True
    )

    print("🎉 학습 완료!")
    return os.path.join(SAVE_DIR, 'yolov8m_damage', 'weights', 'best.pt')

# 학습 실행
best_model_path = train_model()
print(f"🏆 Best Model Path: {best_model_path}")

## 성능 테스트(test)

In [ ]:
# [Cell 3] 성능 평가 (Detection & Classification)

best_model_path = '/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/TRAIN_RESULT/yolov8x_damage_class_all_2nd/weights/best.pt'
RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"
os.makedirs(RESULT_ROOT, exist_ok=True)

def evaluate_performance(model_path):
    if not os.path.exists(model_path):
        print(f"❌ 모델 파일이 없습니다: {model_path}")
        return

    print(f"🚀 평가 시작...")
    print(f"   - 모델: {model_path}")

    model = YOLO(model_path)

    # ---------------------------------------------------------
    # 1. 객체 탐지 성능 평가 (Detection Metrics: mAP)
    # ---------------------------------------------------------
    print("\n📊 [1] BBox 탐지 성능 (mAP) 계산 중...")
    # split='test'로 설정하여 Test Set에 대해 평가
    metrics = model.val(data=YAML_PATH, split='test', verbose=False)

    map50 = metrics.box.map50
    map5095 = metrics.box.map

    # ---------------------------------------------------------
    # 2. 이미지 분류 성능 & 속도 평가 (Classification & Speed)
    # ---------------------------------------------------------
    print("📊 [2] 이미지 분류 성능 및 추론 속도 측정 중...")

    TEST_IMG_DIR = os.path.join(DATASET_ROOT, 'images', 'test')
    TEST_LBL_DIR = os.path.join(DATASET_ROOT, 'labels', 'test')

    image_files = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    results_list = []
    speed_stats = []

    for file in tqdm(image_files, desc="Inference"):
        img_path = os.path.join(TEST_IMG_DIR, file)
        txt_file = os.path.splitext(file)[0] + ".txt"
        lbl_path = os.path.join(TEST_LBL_DIR, txt_file)

        # A. 정답 확인 (라벨 파일 내용 유무)
        true_label = 0
        if os.path.exists(lbl_path) and os.path.getsize(lbl_path) > 0:
            true_label = 1 # Damaged

        # B. 모델 추론
        results = model(img_path, conf=0.25, verbose=False)

        # C. 속도 측정
        speed_info = results[0].speed
        total_time_ms = speed_info['preprocess'] + speed_info['inference'] + speed_info['postprocess']
        speed_stats.append(total_time_ms)

        # D. 예측 라벨 (박스 유무)
        pred_label = 0
        if len(results[0].boxes) > 0:
            pred_label = 1 # Damaged

        results_list.append({
            "filename": file,
            "true_label": true_label,
            "pred_label": pred_label,
            "is_correct": (true_label == pred_label)
        })

    df = pd.DataFrame(results_list)

    # =========================================================
    # [최종 분석 결과 리포트]
    # =========================================================
    print("\n" + "="*50)
    print("📊 [최종 분석 결과 - Fine-tuned Model]")
    print("="*50)

    # 1. mAP 결과 출력
    print(f"🎯 1. 객체 탐지 성능 (Detection Metrics):")
    print(f"   - mAP@50    : {map50:.4f}")
    print(f"   - mAP@50-95 : {map5095:.4f}")

    # 2. 정확도 출력
    acc = accuracy_score(df['true_label'], df['pred_label'])
    print(f"\n✅ 2. 분류 정확도 (Accuracy): {acc:.4f} ({acc*100:.2f}%)")

    # 3. 속도 출력
    if speed_stats:
        avg_time = np.mean(speed_stats)
        min_time = np.min(speed_stats)
        max_time = np.max(speed_stats)
        fps = 1000 / avg_time if avg_time > 0 else 0

        print(f"\n⚡ 3. 추론 속도 (Inference Speed):")
        print(f"   - 평균 소요 시간 : {avg_time:.2f} ms/장")
        print(f"   - 최소 소요 시간 : {min_time:.2f} ms")
        print(f"   - 최대 소요 시간 : {max_time:.2f} ms")
        print(f"   - 처리량 (FPS)   : {fps:.2f} FPS")

    # 4. 상세 리포트
    print(f"\n📝 4. 상세 분류 리포트:")
    print(classification_report(df['true_label'], df['pred_label'], target_names=['Normal', 'Damaged']))

    # 5. 혼동 행렬 시각화
    plt.figure(figsize=(6, 5))
    cm = confusion_matrix(df['true_label'], df['pred_label'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred: Normal', 'Pred: Damaged'],
                yticklabels=['True: Normal', 'True: Damaged'])
    plt.title(f'Confusion Matrix\n(Damage Detection)')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

    # 결과 저장
    save_path = os.path.join(RESULT_ROOT, "inference_damage_test_results_2nd_025.csv")
    df.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 전체 실행 결과 저장 완료: {save_path}")

# 평가 실행
evaluate_performance(best_model_path)


## 오탐 대상 시각화

In [ ]:
# 모델 및 결과 파일 경로
MODEL_PATH = os.path.join(SAVE_DIR, 'yolov8x_damage_class_all_2nd', 'weights', 'best.pt')
RESULT_CSV_PATH = os.path.join(RESULT_ROOT, "inference_damage_test_results_2nd_025.csv")

visualize_failures_batch(start_index=0, batch_size=10)

## 성능 테스트(conf 0.25->0.1)

In [ ]:
# [Cell 3] 성능 평가 (Detection & Classification)

best_model_path = '/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/TRAIN_RESULT/yolov8x_damage_class_all_2nd/weights/best.pt'
RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"
os.makedirs(RESULT_ROOT, exist_ok=True)

def evaluate_performance(model_path):
    if not os.path.exists(model_path):
        print(f"❌ 모델 파일이 없습니다: {model_path}")
        return

    print(f"🚀 평가 시작...")
    print(f"   - 모델: {model_path}")

    model = YOLO(model_path)

    # ---------------------------------------------------------
    # 1. 객체 탐지 성능 평가 (Detection Metrics: mAP)
    # ---------------------------------------------------------
    print("\n📊 [1] BBox 탐지 성능 (mAP) 계산 중...")
    # split='test'로 설정하여 Test Set에 대해 평가
    metrics = model.val(data=YAML_PATH, split='test', verbose=False)

    map50 = metrics.box.map50
    map5095 = metrics.box.map

    # ---------------------------------------------------------
    # 2. 이미지 분류 성능 & 속도 평가 (Classification & Speed)
    # ---------------------------------------------------------
    print("📊 [2] 이미지 분류 성능 및 추론 속도 측정 중...")

    TEST_IMG_DIR = os.path.join(DATASET_ROOT, 'images', 'test')
    TEST_LBL_DIR = os.path.join(DATASET_ROOT, 'labels', 'test')

    image_files = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    results_list = []
    speed_stats = []

    for file in tqdm(image_files, desc="Inference"):
        img_path = os.path.join(TEST_IMG_DIR, file)
        txt_file = os.path.splitext(file)[0] + ".txt"
        lbl_path = os.path.join(TEST_LBL_DIR, txt_file)

        # A. 정답 확인 (라벨 파일 내용 유무)
        true_label = 0
        if os.path.exists(lbl_path) and os.path.getsize(lbl_path) > 0:
            true_label = 1 # Damaged

        # B. 모델 추론
        results = model(img_path, conf=0.1, verbose=False)

        # C. 속도 측정
        speed_info = results[0].speed
        total_time_ms = speed_info['preprocess'] + speed_info['inference'] + speed_info['postprocess']
        speed_stats.append(total_time_ms)

        # D. 예측 라벨 (박스 유무)
        pred_label = 0
        if len(results[0].boxes) > 0:
            pred_label = 1 # Damaged

        results_list.append({
            "filename": file,
            "true_label": true_label,
            "pred_label": pred_label,
            "is_correct": (true_label == pred_label)
        })

    df = pd.DataFrame(results_list)

    # =========================================================
    # [최종 분석 결과 리포트]
    # =========================================================
    print("\n" + "="*50)
    print("📊 [최종 분석 결과 - Fine-tuned Model]")
    print("="*50)

    # 1. mAP 결과 출력
    print(f"🎯 1. 객체 탐지 성능 (Detection Metrics):")
    print(f"   - mAP@50    : {map50:.4f}")
    print(f"   - mAP@50-95 : {map5095:.4f}")

    # 2. 정확도 출력
    acc = accuracy_score(df['true_label'], df['pred_label'])
    print(f"\n✅ 2. 분류 정확도 (Accuracy): {acc:.4f} ({acc*100:.2f}%)")

    # 3. 속도 출력
    if speed_stats:
        avg_time = np.mean(speed_stats)
        min_time = np.min(speed_stats)
        max_time = np.max(speed_stats)
        fps = 1000 / avg_time if avg_time > 0 else 0

        print(f"\n⚡ 3. 추론 속도 (Inference Speed):")
        print(f"   - 평균 소요 시간 : {avg_time:.2f} ms/장")
        print(f"   - 최소 소요 시간 : {min_time:.2f} ms")
        print(f"   - 최대 소요 시간 : {max_time:.2f} ms")
        print(f"   - 처리량 (FPS)   : {fps:.2f} FPS")

    # 4. 상세 리포트
    print(f"\n📝 4. 상세 분류 리포트:")
    print(classification_report(df['true_label'], df['pred_label'], target_names=['Normal', 'Damaged']))

    # 5. 혼동 행렬 시각화
    plt.figure(figsize=(6, 5))
    cm = confusion_matrix(df['true_label'], df['pred_label'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred: Normal', 'Pred: Damaged'],
                yticklabels=['True: Normal', 'True: Damaged'])
    plt.title(f'Confusion Matrix\n(Damage Detection)')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

    # 결과 저장
    save_path = os.path.join(RESULT_ROOT, "inference_damage_test_results_2nd_010.csv")
    df.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 전체 실행 결과 저장 완료: {save_path}")

# 평가 실행
evaluate_performance(best_model_path)


In [ ]:
# 모델 및 결과 파일 경로
MODEL_PATH = os.path.join(SAVE_DIR, 'yolov8x_damage_class_all_2nd', 'weights', 'best.pt')
RESULT_CSV_PATH = os.path.join(RESULT_ROOT, "inference_damage_test_results_2nd_010.csv")

visualize_failures_batch(start_index=0, batch_size=6)